# 5. Geospatial Analysis

In [ ]:
import pandas as pd
import folium
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
# Load the shapefile
POA_data = gpd.read_file('../data/external/POA_2021_AUST_GDA2020_SHP')
print(POA_data.head())

In [3]:
income_per_postcode = pd.read_csv('../data/external/income_per_postcode.csv')

### 5.1 Process of external dataset 

In [4]:
income_per_postcode['Average taxable income or loss'] = income_per_postcode['Average taxable income or loss'].str.replace(',', '').astype(int)
income_per_postcode['Postcode'] = income_per_postcode['Postcode'].astype(str)
income_per_postcode = income_per_postcode[['Postcode', 'Average taxable income or loss']]

In [5]:
POA_data['POA_CODE21'] = POA_data['POA_CODE21'].astype(str)

In [ ]:
income_per_postcode

### 5.2 Data visualization by postcode

In [ ]:
# Connect the document 
merged_data = POA_data.merge(income_per_postcode, left_on='POA_CODE21', right_on='Postcode')

# convert the dollar value into per thousand unit 
merged_data['Average taxable income or loss'] = merged_data['Average taxable income or loss'].astype(float) / 1000

# create map 
m = folium.Map(location=[-25.2744, 133.7751], zoom_start=4)

# add to folium
folium.Choropleth(
    geo_data=merged_data,
    name='choropleth',
    data=merged_data,
    columns=['Postcode', 'Average taxable income or loss'],
    key_on='feature.properties.Postcode',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Income (Thousands AUD)'
).add_to(m)


folium.LayerControl().add_to(m)

# show map
m

In [8]:
data = pd.read_csv('../data/curated/external/dollar_amount_per_postcode.csv')

### 5.3 Data visualization by SA2

In [ ]:
# Convert both columns to string
POA_data['POA_CODE21'] = POA_data['POA_CODE21'].astype(str)
data['postcode'] = data['postcode'].astype(str)

# Now, try merging again
merged_data = POA_data.merge(data, left_on='POA_CODE21', right_on='postcode')
fig, ax = plt.subplots(figsize=(15,15))
merged_data.plot(column='dollar_value', legend=True, ax=ax, cmap='OrRd', edgecolor="k", linewidth=0.2)
ax.set_title("Dollar Value by Postcode")
plt.show() 

In [ ]:
# Convert your merged data back to a GeoDataFrame
gdf = gpd.GeoDataFrame(merged_data, geometry='geometry')

# Create a folium map object
m = folium.Map(
    location=[-25.2744, 133.7751],  # This centers the map to Australia
    tiles='CartoDB positron',
    zoom_start=5
)

# Add the choropleth layer
folium.Choropleth(
    geo_data=gdf,
    name="choropleth",
    data=gdf,
    columns=["POA_CODE21", "dollar_value"],
    key_on="feature.properties.POA_CODE21",
    fill_color="OrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Total Dollar Value by Postcode"
).add_to(m)

# Add layer control to turn choropleth on or off (useful if you have other layers)
folium.LayerControl().add_to(m)

# Show the map
m